In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool
import smtplib
#import openai
#from langfuse.openai import openai
from langfuse.openai import openai
from email.message import EmailMessage
import os
from langfuse import Langfuse, observe
import asyncio
import pprint
from pydantic import BaseModel


load_dotenv(override=True)

lf = Langfuse()

SENDER_EMAIL = os.getenv('EMAIL_USER')
APP_PASSWORD = os.getenv('EMAIL_PASSWORD')
gemini_model = os.getenv('GEMINI_MODEL')
openai_api_key = os.getenv('GOOGLE_API_KEY')
openai_url = os.getenv('GEMINI_BASE_URL')
RECIPIENT_EMAIL = SENDER_EMAIL

openai_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = AsyncOpenAI(
    api_key=openai_api_key,
    base_url=openai_url
)

gemini=OpenAIChatCompletionsModel(
        model=gemini_model,
        openai_client=gemini_client,
)

### How to create an agent:

In [ ]:
agent = Agent(
    name="Jokester", 
    instructions="You are a joke teller", 
    model=gemini
)

In [ ]:
result = await Runner.run(agent, "Tell me a really good joke")

In [ ]:
class EmailBody(BaseModel):
    body:str

no_subject="You do not create a subject for the email"

instructions1 = f"You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails. \
    {no_subject}"

instructions2 = f"You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response. \
    {no_subject}"

instructions3 = f"You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails. \
    {no_subject}"

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=gemini,
        output_type=EmailBody
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=gemini,
        output_type=EmailBody
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=gemini,
        output_type=EmailBody
)

How to stream the results:

In [ ]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, openai.types.responses.ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
async def parallel_call(prompt):
    with lf.start_as_current_span(name="call-all-sales-with-same-prompt") as span:
        results = await asyncio.gather(
            Runner.run(sales_agent1, input=prompt),
            Runner.run(sales_agent2, input=prompt),
            Runner.run(sales_agent3, input=prompt)
        )
    return results

results = await parallel_call("Write a cold sales email")

In [ ]:

for result in results:
    print("Result:")
    print(result.final_output)
    print("********")

In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=gemini,
    output_type=EmailBody
)

In [ ]:
async def get_best_cold_salesman(results):
    with lf.start_as_current_span(name="sales-picker") as span:
        prompt = "Options:"
        for index,result in enumerate(results):
            prompt += f"Sales Agent {index}:\n {result.final_output}\n\n"
        return await Runner.run(sales_picker, prompt)

best = await get_best_cold_salesman(results)

In [ ]:
print(best.final_output)

In [ ]:
@function_tool
def send_email(body:str, subject:str=""):
    """This tool sends an email, received a body and a subject (if present)"""
    # --- Create the Email Message ---
    msg = EmailMessage()
    msg['Subject'] = f"Sales Agent - {subject}"
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECIPIENT_EMAIL
    msg.set_content(body)

    # --- Send the Email ---
    try:
        # Connect to the SMTP server (for Gmail)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)  # Log in to the server
            smtp.send_message(msg)                 # Send the email
            print("Email sent successfully!")
    except Exception as e:
        print(f"An error occurred: {e.with_traceback}")


In [ ]:
pprint.pprint(send_email)

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
pprint.pprint(tool1)

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write a cold sales email")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a cold sales email")

tools = [tool1, tool2, tool3, send_email]

tools

In [ ]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools and send this email as the body.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and you must choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email you selected (and only the best email) to the user. The best email is sent as the body for the send_email function
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model=gemini)

message = "Send a cold sales email addressed to 'Dear CEO'"



In [ ]:
with lf.start_as_current_span(name="sales-manager") as span:
    result = await Runner.run(sales_manager, message)

In [ ]:
pprint.pprint(result.final_output)

Now we will create an handoff for the previous Agent.
This handoff will be a Agent with tree tools:
 - Create a subject
 - Format the email into html
 - send the email

In [ ]:
@function_tool
def send_email_html(html_body:str, subject:str):
    """This tool sends an email, received a html email body and the subject of the email"""
    # --- Create the Email Message ---
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECIPIENT_EMAIL
    msg.add_alternative(html_body, subtype='html')
    # --- Send the Email ---
    try:
        # Connect to the SMTP server (for Gmail)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)  # Log in to the server
            smtp.send_message(msg)                 # Send the email
            print("Email sent successfully!")
    except Exception as e:
        print(f"An error occurred: {e.with_traceback}")

In [ ]:
class EmailSubject(BaseModel):
    subject:str

subject_prompt = "You can write a subject for a cold sales email. \
You are given a message and you need to write one subject for an email that is likely to get a response. \
Your response will only have one subject you'll say: The best subject is:<the one subject you created> "

format_email_to_html= "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_agent = Agent(
    name="Subject agent",
    instructions=subject_prompt,
    model=gemini,
    output_type=EmailSubject
).as_tool(
    tool_name="Create_subject",
    tool_description="Receives an email and creates a subject for it"
)

format_email_to_html = Agent(
    name="Format email to html",
    instructions=format_email_to_html,
    model=gemini,
    output_type=EmailBody
).as_tool(
    tool_name="Format_email_to_html",
    tool_description="received an email and formats it into html"
)

tools1 = [subject_agent, format_email_to_html, send_email_html]

In [ ]:
pprint.pprint(subject_agent)

In [ ]:
email_sender = """
You are an efficient email-processing agent. Your sole objective is to take an email body, generate a subject, format the body into HTML, and send the email.

You will perform the following steps in this exact order:

    1. You will be given an email body in a json object.

    2. You will execute the following two tool calls in parallel:

        1. subject_agent: Use this tool with the original email body to generate an appropriate subject line.

        2. format_email_to_html: Use this tool with the original email body to convert it into HTML.

    3. Once you have received the subject from subject_agent and the HTML from format_email_to_html, you will call the send_email_html tool once, providing it with the generated subject and HTML body.
"""

email_processor_agent = Agent(
    name="Orchestrate email processing",
    instructions=email_sender,
    tools = tools1,
    model=gemini,
    handoff_description="Convert an email to HTML create the subeject and send it"
)

In [ ]:
tools=[tool1,tool2,tool3]
handoffs=[email_processor_agent]

In [ ]:
email_creator_instructions = """
You are a discerning Sales Manager at ComplAI. Your primary role is to ensure that all outgoing sales communications are of the highest quality and perfectly aligned with our brand. You are meticulous, strategic, and an excellent judge of effective sales copy.

Your objective is to select the most effective sales email from a set of drafts and pass it on for processing.

You will follow these steps precisely:

    1. You will receive a request to generate a sales email.

    2. Immediately call the three sales_agent tools in parallel, each with the same request details. Each tool will provide you with one distinct email draft, resulting in three total options.

    3. Critically evaluate the three drafts based on the following criteria:

        - Clarity: Is the value proposition clear and easy to understand in under 10 seconds?

        - Tone: Does the email sound like it's from a helpful, professional expert at ComplAI? Avoid overly aggressive or generic marketing language.

        - Call to Action (CTA): Is the next step for the customer obvious, compelling, and easy to execute?

    4. Based on your evaluation, choose the single best email that most effectively meets all the criteria.

    5. Once you have made your selection, handoff the flow to the 'Orchestrate email processing' agent, providing only the chosen email's content. Do not send the email yourself. Your task is complete after the handoff.
"""

email_body_creator=Agent(
    name="Create and choose the best cold email",
    instructions=email_creator_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini
)

In [ ]:
with lf.start_as_current_span(name="body_creator") as span:
    result = await Runner.run(email_body_creator, "Send a cold email, from André to Vizinha")